In [2]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🧠 Neural Spike Analysis: Rate vs Temporal Coding\n",
    "\n",
    "This notebook analyzes neural spike data to determine whether information is encoded in **firing rates** or **temporal patterns**.\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import libraries\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from scipy import stats\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Import our custom modules\n",
    "import sys\n",
    "sys.path.append('../src')\n",
    "\n",
    "from spike_data_loader import SpikeDataLoader\n",
    "from spike_analyzer import SpikeAnalyzer\n",
    "from spike_visualizer import SpikeVisualizer\n",
    "from decoding_analysis import DecodingAnalysis\n",
    "\n",
    "print(\"✅ All modules imported successfully!\")\n",
    "print(\"🧠 Ready for neural analysis!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize analysis modules\n",
    "loader = SpikeDataLoader()\n",
    "analyzer = SpikeAnalyzer()\n",
    "visualizer = SpikeVisualizer(style='dark')\n",
    "decoder = DecodingAnalysis()\n",
    "\n",
    "print(\"🔧 Analysis modules initialized!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📂 Load Data\n",
    "\n",
    "**IMPORTANT**: If you've generated data using the data generator, uncomment and use the first option. Otherwise, we'll generate synthetic data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ===== OPTION 1: Load generated data files (RECOMMENDED) =====\n",
    "# Uncomment these lines if you've run the data generator:\n",
    "\n",
    "# spike_data = loader.load_spike_data('../data/neural_data_mixed_coding.npz')\n",
    "# spike_times = spike_data['spike_times']\n",
    "# unit_ids = spike_data['unit_ids']\n",
    "# event_times = spike_data['event_times']\n",
    "# event_labels = spike_data['event_labels']\n",
    "# print(\"✅ Loaded data from file!\")\n",
    "\n",
    "# ===== OPTION 2: Generate data on-the-fly =====\n",
    "# Use this if you haven't generated data files yet\n",
    "\n",
    "print(\"📊 Generating synthetic data...\")\n",
    "synthetic_data = loader.create_synthetic_data(\n",
    "    n_trials=80,\n",
    "    n_units=10,\n",
    "    duration=4.0,\n",
    "    stimulus_effect=2.0\n",
    ")\n",
    "\n",
    "spike_times = synthetic_data['spike_times']\n",
    "unit_ids = synthetic_data['unit_ids']\n",
    "event_times = synthetic_data['event_times']\n",
    "event_labels = synthetic_data['event_labels']\n",
    "\n",
    "print(f\"✅ Data ready:\")\n",
    "print(f\"   • {len(np.unique(unit_ids))} units\")\n",
    "print(f\"   • {len(event_times)} trials\")\n",
    "print(f\"   • {len(spike_times)} total spikes\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create trial structure\n",
    "print(\"🔄 Creating trial-aligned data...\")\n",
    "\n",
    "trials_data = loader.create_trials(\n",
    "    spike_times=spike_times,\n",
    "    unit_ids=unit_ids,\n",
    "    event_times=event_times,\n",
    "    event_labels=event_labels,\n",
    "    pre_time=1.0,\n",
    "    post_time=3.0\n",
    ")\n",
    "\n",
    "print(f\"✅ Created {len(trials_data)} trials\")\n",
    "\n",
    "# Show data summary\n",
    "summary = loader.get_data_summary(trials_data)\n",
    "print(\"\\n📊 Data Summary:\")\n",
    "for key, value in summary.items():\n",
    "    print(f\"   • {key}: {value}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 Quality Control"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze unit quality\n",
    "print(\"🔍 Quality control analysis...\")\n",
    "\n",
    "unit_stats = analyzer.calculate_unit_stats(trials_data)\n",
    "good_units = analyzer.filter_good_units(\n",
    "    unit_stats,\n",
    "    min_firing_rate=0.1,\n",
    "    max_refractory_violations=0.02,\n",
    "    min_total_spikes=50\n",
    ")\n",
    "\n",
    "print(f\"✅ {len(good_units)}/{len(unit_stats)} units passed QC\")\n",
    "print(f\"Good units: {good_units}\")\n",
    "\n",
    "# Visualize quality\n",
    "fig = visualizer.plot_unit_quality(unit_stats, good_units)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Spike Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Pick a good unit for visualization\n",
    "if good_units:\n",
    "    example_unit = good_units[0]\n",
    "else:\n",
    "    example_unit = list(unit_stats.keys())[0]\n",
    "    print(\"⚠️ Using first available unit (didn't pass QC)\")\n",
    "\n",
    "print(f\"📊 Visualizing Unit {example_unit}...\")\n",
    "\n",
    "# Beautiful raster plot and PSTH\n",
    "fig = visualizer.plot_raster_psth(trials_data, example_unit)\n",
    "plt.show()\n",
    "\n",
    "# ISI analysis\n",
    "fig = visualizer.plot_isi_distribution(trials_data, example_unit)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📈 Find Responsive Units"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test which units respond to stimulus\n",
    "print(\"📈 Testing for stimulus-responsive units...\")\n",
    "\n",
    "responsive_units = []\n",
    "response_stats = {}\n",
    "\n",
    "for unit_id in good_units:\n",
    "    # Compare baseline vs stimulus periods\n",
    "    baseline_rates, stim_rates = analyzer.get_baseline_vs_stimulus_rates(\n",
    "        trials_data, unit_id,\n",
    "        baseline_window=(-1.0, 0.0),\n",
    "        stim_window=(0.0, 2.0)\n",
    "    )\n",
    "    \n",
    "    # Statistical test\n",
    "    try:\n",
    "        stat, p_value = stats.wilcoxon(baseline_rates, stim_rates, alternative='two-sided')\n",
    "    except:\n",
    "        p_value = 1.0\n",
    "    \n",
    "    response_stats[unit_id] = {\n",
    "        'baseline_rate': np.mean(baseline_rates),\n",
    "        'stimulus_rate': np.mean(stim_rates),\n",
    "        'p_value': p_value,\n",
    "        'responsive': p_value < 0.05\n",
    "    }\n",
    "    \n",
    "    if p_value < 0.05:\n",
    "        responsive_units.append(unit_id)\n",
    "        print(f\"   ✅ Unit {unit_id}: responsive (p = {p_value:.4f})\")\n",
    "    else:\n",
    "        print(f\"   ❌ Unit {unit_id}: not responsive (p = {p_value:.4f})\")\n",
    "\n",
    "print(f\"\\n📊 {len(responsive_units)}/{len(good_units)} units are responsive\")\n",
    "\n",
    "# Visualize responses\n",
    "if response_stats:\n",
    "    fig = visualizer.plot_response_statistics(response_stats)\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎯 MAIN ANALYSIS: Rate vs Temporal Coding\n",
    "\n",
    "This is the core analysis! We'll compare how well we can decode stimulus type using:\n",
    "- **Rate features**: Spike counts\n",
    "- **Temporal features**: Precise timing patterns"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Select units for decoding\n",
    "analysis_units = responsive_units if responsive_units else good_units[:5]\n",
    "\n",
    "if not analysis_units:\n",
    "    analysis_units = list(unit_stats.keys())[:5]\n",
    "    print(\"⚠️ Using any available units\")\n",
    "\n",
    "print(f\"🎯 Decoding with {len(analysis_units)} units: {analysis_units}\")\n",
    "\n",
    "# Extract features\n",
    "print(\"\\n📊 Extracting features...\")\n",
    "\n",
    "# Rate features (spike counts)\n",
    "rate_features = decoder.extract_rate_features(\n",
    "    trials_data, analysis_units, time_window=(0.0, 2.0)\n",
    ")\n",
    "\n",
    "# Temporal features (binned spike patterns)\n",
    "temporal_features = decoder.extract_temporal_features(\n",
    "    trials_data, analysis_units, time_window=(0.0, 2.0), bin_size=0.1\n",
    ")\n",
    "\n",
    "# Get trial labels\n",
    "labels = decoder.get_trial_labels(trials_data)\n",
    "\n",
    "print(f\"✅ Rate features: {rate_features.shape}\")\n",
    "print(f\"✅ Temporal features: {temporal_features.shape}\")\n",
    "print(f\"✅ Labels: {np.unique(labels, return_counts=True)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compare decoding performance\n",
    "print(\"🎯 Running decoding analysis...\")\n",
    "\n",
    "results = decoder.compare_decoding_performance(\n",
    "    rate_features=rate_features,\n",
    "    temporal_features=temporal_features,\n",
    "    labels=labels,\n",
    "    n_folds=5\n",
    ")\n",
    "\n",
    "print(\"\\n🎯 DECODING RESULTS:\")\n",
    "print(f\"   Rate accuracy:     {results['rate_accuracy']:.3f} ± {results['rate_std']:.3f}\")\n",
    "print(f\"   Temporal accuracy: {results['temporal_accuracy']:.3f} ± {results['temporal_std']:.3f}\")\n",
    "print(f\"   Shuffle control:   {results['shuffle_accuracy']:.3f} ± {results['shuffle_std']:.3f}\")\n",
    "print(f\"   Temporal AUC:      {results['temporal_auc']:.3f}\")\n",
    "\n",
    "# Statistical significance\n",
    "if results['p_value_rate_vs_temporal'] < 0.05:\n",
    "    significance = \"SIGNIFICANT\"\n",
    "else:\n",
    "    significance = \"not significant\"\n",
    "    \n",
    "print(f\"\\n📊 Rate vs Temporal: p = {results['p_value_rate_vs_temporal']:.4f} ({significance})\")\n",
    "\n",
    "# Visualize results\n",
    "fig = visualizer.plot_decoding_results(results)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔬 Advanced Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Cross-correlation between units\n",
    "if len(analysis_units) >= 2:\n",
    "    print(f\"🔗 Cross-correlation: Unit {analysis_units[0]} vs {analysis_units[1]}\")\n",
    "    fig = visualizer.plot_cross_correlation(trials_data, analysis_units[0], analysis_units[1])\n",
    "    plt.show()\n",
    "\n",
    "# Individual unit performance\n",
    "print(\"\\n🔍 Individual unit decoding:\")\n",
    "individual_results = decoder.analyze_individual_units(trials_data, analysis_units[:3])\n",
    "\n",
    "for unit_id, result in individual_results.items():\n",
    "    improvement = result['improvement']\n",
    "    print(f\"   Unit {unit_id}: Rate={result['rate_accuracy']:.3f}, Temporal={result['temporal_accuracy']:.3f}, Δ={improvement:+.3f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📋 Final Results & Conclusions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate comprehensive report\n",
    "report = decoder.create_decoding_report(results, analysis_units)\n",
    "print(report)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Final interpretation\n",
    "rate_acc = results['rate_accuracy']\n",
    "temp_acc = results['temporal_accuracy']\n",
    "improvement = temp_acc - rate_acc\n",
    "\n",
    "print(\"\\n\" + \"=\"*60)\n",
    "print(\"🎯 FINAL CONCLUSIONS\")\n",
    "print(\"=\"*60)\n",
    "\n",
    "print(f\"\\n📊 Performance Summary:\")\n",
    "print(f\"   • Rate-based decoding: {rate_acc:.1%}\")\n",
    "print(f\"   • Temporal decoding: {temp_acc:.1%}\")\n",
    "print(f\"   • Improvement: {improvement:+.1%}\")\n",
    "\n",
    "print(f\"\\n🧠 Coding Strategy:\")\n",
    "if improvement > 0.05 and results['p_value_rate_vs_temporal'] < 0.05:\n",
    "    print(f\"   🎯 TEMPORAL CODING DETECTED!\")\n",
    "    print(f\"   💡 Neurons encode information in precise spike timing\")\n",
    "    print(f\"   🔬 Focus on synchrony and temporal dynamics\")\n",
    "elif improvement < -0.05 and results['p_value_rate_vs_temporal'] < 0.05:\n",
    "    print(f\"   📊 RATE CODING DETECTED!\")\n",
    "    print(f\"   💡 Neurons encode information in firing rates\")\n",
    "    print(f\"   🔬 Spike counts are sufficient\")\n",
    "else:\n",
    "    print(f\"   🤔 MIXED/UNCLEAR CODING\")\n",
    "    print(f\"   💡 Both rate and timing may contribute\")\n",
    "    print(f\"   🔬 Consider hybrid approaches\")\n",
    "\n",
    "print(f\"\\n📈 Dataset Quality:\")\n",
    "print(f\"   • Good units: {len(good_units)}/{len(unit_stats)}\")\n",
    "print(f\"   • Responsive units: {len(responsive_units)}\")\n",
    "print(f\"   • Total trials: {len(trials_data)}\")\n",
    "\n",
    "if temp_acc > 0.7:\n",
    "    print(f\"\\n✨ Excellent decoding performance!\")\n",
    "elif temp_acc > 0.6:\n",
    "    print(f\"\\n👍 Good decoding performance\")\n",
    "else:\n",
    "    print(f\"\\n💡 Consider improving data quality or increasing trials\")\n",
    "\n",
    "print(\"\\n🧠 Analysis Complete! 🎉\")\n",
    "print(\"=\"*60)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n",
    "\n",
    "## 🚀 Next Steps\n",
    "\n",
    "1. **Try different datasets**: If you generated multiple data files, load different ones to see how results change\n",
    "2. **Adjust parameters**: Change time windows, bin sizes, etc.\n",
    "3. **Add more units**: Include more units in the analysis\n",
    "4. **Extend analysis**: Add population-level metrics, more sophisticated decoders\n",
    "\n",
    "**Congratulations on completing your neural coding analysis! 🧠✨**"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined